In [1]:
### Compile list of instruments:
### - all currently deployed (entries with blank end dates in the deployment sheet)
### - all deployed in targetYear
### - all recovered in targetYear


In [2]:
# -*- coding: utf-8 -*-

In [3]:
targetYear = 2022

In [4]:
from os import path
import pandas as pd
import os
import numpy as np
import datetime

In [5]:
### Deployment Files
InstDeploy = path.dirname('/Users/wruef/githubRepos/asset-management/deployment/')
outputFile_deployed = 'reportOuts/deployedInstruments_' + str(targetYear) + '.csv'
outputFile_recovered = 'reportOuts/recoveredInstruments_' + str(targetYear) + '.csv'
outputFile_current = 'reportOuts/currentDeployments_' + str(targetYear) + '.csv'

In [6]:
### Load Instrument deployments and filter for empty end times

CabledArray = pd.Series(['CE02SHBP','CE04OSBP','CE04OSPD','CE04OSPS','RS01SBPD','RS01SBPS',
                        'RS01SLBS','RS01SUM1','RS03AXBS','RS03AXPD','RS03AXPS','RS03INT2',
                        'RS03INT1','RS01SUM2','RS03CCAL','RS03ECAL','RS03ASHS'])
deploymentSheets = []
for array in CabledArray:
    deployFilePath = InstDeploy + '/' + array + '_Deploy.csv' 
    deploymentSheets.append(deployFilePath)
    
df = pd.concat([pd.read_csv(f, skip_blank_lines = True, comment='#') for f in deploymentSheets], ignore_index = True)


In [7]:
### load in instrument list
instrumentFile = 'params/RCA-InstrumentList.csv'
assetList = pd.read_csv(instrumentFile)

asset_dict = assetList.set_index('assetID').T.to_dict('series')

In [8]:
df_current = df.loc[df['stopDateTime'].isnull()]

In [9]:
df_recovered = df.loc[df['stopDateTime'].notnull()]
df_recovered['recoverYear'] = pd.to_datetime(df_recovered['stopDateTime']).dt.year
df_recovered_year = df_recovered[df_recovered['recoverYear'] == targetYear]


/var/folders/cy/xccs89nn3dv_w5j017h33t580000gn/T/ipykernel_17609/1756926738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recovered['recoverYear'] = pd.to_datetime(df_recovered['stopDateTime']).dt.year


In [10]:
df['deployYear'] = pd.to_datetime(df['startDateTime']).dt.year
df_deployed_year = df[df['deployYear'] == targetYear]


In [11]:
with open(outputFile_current,'w') as f:
    f.write('referenceDesignator,Cruise,instrumentType,deployDate,assetID,serialNumber\n')
    for index,deployment in df_current.iterrows():
        f.write("%s,%s,%s,%s,%s,%s\n" % (deployment['Reference Designator'],
                                      deployment['CUID_Deploy'],
                                      asset_dict[deployment['sensor.uid']]['instrumentType'],
                                      deployment['startDateTime'], deployment['sensor.uid'],
                                      asset_dict[deployment['sensor.uid']]['mfgSN']))
        

In [12]:
with open(outputFile_deployed,'w') as f:
    f.write('referenceDesignator,Cruise,instrumentType,deployDate,assetID,serialNumber\n')
    for index,deployment in df_deployed_year.iterrows():
        f.write("%s,%s,%s,%s,%s,%s\n" % (deployment['Reference Designator'],
                                      deployment['CUID_Deploy'],
                                      asset_dict[deployment['sensor.uid']]['instrumentType'],
                                      deployment['startDateTime'], deployment['sensor.uid'],
                                      asset_dict[deployment['sensor.uid']]['mfgSN']))

In [13]:
with open(outputFile_recovered,'w') as f:
    f.write('referenceDesignator,Cruise,instrumentType,recoverDate,assetID,serialNumber\n')
    for index,deployment in df_recovered_year.iterrows():
        f.write("%s,%s,%s,%s,%s,%s\n" % (deployment['Reference Designator'],
                                      deployment['CUID_Deploy'],
                                      asset_dict[deployment['sensor.uid']]['instrumentType'],
                                      deployment['stopDateTime'], deployment['sensor.uid'],
                                      asset_dict[deployment['sensor.uid']]['mfgSN']))
        